In [2]:
import ee
ee.Initialize()
image = ee.Image('srtm90_v4')
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'version': 1494271934303000.0, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [3]:
from ee import batch
image = ee.Image('LANDSAT/LC8_L1T/LC80440342014077LGN00').select('B8')
landsat = ee.Image('LANDSAT/LC8_L1T_TOA/LC81230322014135LGN00').select(['B4', 'B3', 'B2'])
#output the landsat true colour RGB, region defined - Region has to be a nested loop
out1 = batch.Export.image.toDrive(landsat, description='out1', region=([116.2621, 39.8412], [116.2621, 40.01236],[116.4849, 40.01236],[116.4849, 39.8412]))
canny =  ee.Algorithms.CannyEdgeDetector(image,10,1)
out2 = batch.Export.image.toDrive(canny, description='out2', maxPixels=238523062)
process = batch.Task.start(out1)
process = batch.Task.start(out2)

AttributeError: 'Task' object has no attribute 'image'

In [14]:
## define your collection
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')


## Above commented out as you might not know the path and row this allows you to pass a coordinate
pointamazon = ee.Geometry.Point(-3.4653, -62.2159)
pathrow = collection.filterBounds(pointUAE)
 
##Filter cloudy scenes.
clouds = pathrow.filter(ee.Filter.lt('CLOUD_COVER', 5))

## select the bands, we are going for true colour... but could be any!
bands = clouds.select(['B4', 'B3', 'B2'])

##make the data 8-bit.
def convertBit(image):
    return image.multiply(512).uint8()  

## call the conversion    
outputVideo = bands.map(convertBit)

print ("about to build video")
out = batch.Export.video.toDrive(outputVideo, description='negative_ycoordinates', dimensions = 720, framesPerSecond = 2, region=([-3.3,-61], [-3.3,-63],[-3.5,-61],[-3.5,-63]), maxFrames=10000)

## process the image
process = batch.Task.start(out)

print ("process sent to cloud")

about to build video
process sent to cloud
